In [1]:
import configparser
import requests
import json
import pickle
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np
from constants_funcs import int_or_float_or_str, EmbeddingType, QuestionCategory
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /Users/tyler/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
def generate_embeddings(data: pd.DataFrame, column: str, embeddingType: EmbeddingType):
    if (embeddingType == EmbeddingType.Sentence):
        sentence_tokens = [word_tokenize(str(sentence).lower()) for sentence in data[column]]
        tagged_data = [TaggedDocument(words=st, tags=[str(i)]) for i, st in enumerate(sentence_tokens)]
        max_epochs = 100
        vec_size = 20
        alpha = 0.025

        model = Doc2Vec(vector_size=vec_size,
                        alpha=alpha, 
                        min_alpha=0.00025,
                        min_count=1,
                        dm =1,
                        epochs=40)
        
        model.build_vocab(tagged_data)
        model.save(f"emb_{embeddingType.name}_{column}_d2v.model")
        print("Model Saved")
        sentence_vectors = [model.infer_vector(sentence) for sentence in sentence_tokens]
        # auto-save new vectors as column in dataframe so user doesnt have to
        data[f'{column}_{embeddingType.name}_emb'] = sentence_vectors
        return sentence_vectors
    elif embeddingType == EmbeddingType.Word:
        print('word embedding not implemented yet')
    else:
        print('pick a valid embedding enum type')

In [4]:
# example embedding turning question history's question and answer columns into embeddings.
fqh = pd.read_csv('./FULL_QUESTION_HISTORY.csv')
fqh.drop('Unnamed: 0', axis=1, inplace=True)
fqh_q_vecs = generate_embeddings(fqh, 'question', EmbeddingType.Sentence)
fqh_a_vecs = generate_embeddings(fqh, 'answer', EmbeddingType.Sentence)
fqh.to_csv('./FULL_QUESTION_HISTORY_EMBEDDINGS.csv')

Model Saved
Model Saved
